# Quickstart: Install and Run a Minimal Demo

This notebook installs the `snslib` library and runs a very small demonstration of the Stretch and Squeeze experiment.

## 1. Installation

First, ensure you have the prerequisites:
*   Python 3.10+
*   `pip` package manager
*   Git (if installing dependencies like robustness/robustbench)
*   (Optional, for GUI display) `tkinter` (`sudo apt-get install python3-tk` on Debian/Ubuntu)

Run the following cell to install `snslib` and its dependencies from your cloned repository root. You might need to install PyTorch separately based on your CUDA/CPU setup (`pip install torch torchvision ...`).

In [ ]:
# Install the library from the current directory (repository root)
%pip install .

# Optional: Explicit PyTorch install (choose CPU or correct CUDA version)
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

## 2. Minimal Demo (Stretch and Squeeze)

This cell runs a tiny version of the Stretch and Squeeze experiment. 

**IMPORTANT:** You MUST edit the `WEIGHTS_DIR`, `REFERENCES_DIR`, `OUTPUT_DIR`, `CUSTOM_WEIGHTS_DIR` and `NATURAL_RECORDINGS_DIR` variables below to point to the correct locations on your system before running the cell.

In [ ]:
import os
import numpy as np
import torch
import warnings

# Import core components
from snslib.core.utils.parameters import ArgParams, ParamConfig
from snslib.experiment.utils.args import ExperimentArgParams
from snslib.experiment.stretch_and_squeeze import StretchSqueezeExperiment # Hero experiment

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# --- User Parameters (EDIT THESE PATHS!) ---
# !! These paths MUST be set correctly for the demo to run !!
WEIGHTS_DIR = '/path/to/your/DeePSiM' # Placeholder: Path to DeePSiM weights
REFERENCES_DIR = '/path/to/your/references.pkl' # Placeholder: Path to reference code file
OUTPUT_DIR = './output/quickstart_sns' # Where results will be saved (relative path is okay here)
CUSTOM_WEIGHTS_DIR = '/path/to/your/custom_weights' # Placeholder: Path to robust models etc.
NATURAL_RECORDINGS_DIR = '/path/to/your/natural_recordings.pkl' # Placeholder: Path to natural recordings
DATASET_DIR = '/path/to/your/dataset/miniimagenet' # Placeholder: Path to MiniImageNet (not strictly needed for this minimal run if template='T')

# --- Minimal Configuration ---
quickstart_config: ParamConfig = {
    # Experiment Info
    ArgParams.ExperimentName.value: "quickstart_sns",
    ArgParams.ExperimentVersion.value: 0,
    ArgParams.OutputDirectory.value: OUTPUT_DIR,
    ArgParams.NumIterations.value: 3, # Very few iterations
    ArgParams.RandomSeed.value: 42,
    ArgParams.Render.value: False, # No GUI display for quickstart

    # Generator
    ExperimentArgParams.GenWeights.value: WEIGHTS_DIR,
    ExperimentArgParams.GenVariant.value: "fc7",

    # Natural Images (Disabled for simplicity)
    ExperimentArgParams.Template.value: "T", # Only synthetic images
    ExperimentArgParams.Dataset.value: DATASET_DIR, # Path needed, but not used if template='T'
    ExperimentArgParams.Shuffle.value: False,
    ExperimentArgParams.BatchSize.value: 2,
    
    # Natural Stats (Placeholders, not used if template='T')
    ExperimentArgParams.Nat_recs.value: NATURAL_RECORDINGS_DIR,
    ExperimentArgParams.Nrec_aggregate.value: 'max',

    # Subject (Using ResNet50, non-robust for simplicity)
    ExperimentArgParams.NetworkName.value: 'resnet50',
    ExperimentArgParams.RecordingLayers.value: "26=[], 56=[19]", # Example layers
    ExperimentArgParams.CustomWeightsPath.value: CUSTOM_WEIGHTS_DIR,
    ExperimentArgParams.CustomWeightsVariant.value: '', # Empty means non-robust
    ExperimentArgParams.WeightLoadFunction.value: 'torch_load_pretrained',
    ExperimentArgParams.Rec_low.value: "",
    ExperimentArgParams.Rec_high.value: "",
    
    # Scorer
    ExperimentArgParams.ScoringLayers.value: "26=[], 56=[19]", # Must match recording for this setup
    ExperimentArgParams.Reference.value: REFERENCES_DIR,
    ExperimentArgParams.ReferenceInfo.value: "G=fc7, L=56, N=[19], S=1", # Example reference unit
    ExperimentArgParams.ScoringSignature.value: "26=-1, 56=1", # Example: Invariance task
    ExperimentArgParams.Bounds.value: "26=N, 56=N", # Example: No bounds
    ExperimentArgParams.Distance.value: "euclidean",
    ExperimentArgParams.UnitsReduction.value: "mean",
    ExperimentArgParams.LayerReduction.value: "mean",
    ExperimentArgParams.Within_pareto_order.value: 'random',   # LInv
    ExperimentArgParams.Score_low.value: "",
    ExperimentArgParams.Score_high.value: "",
    
    # Optimizer
    ExperimentArgParams.OptimType.value: 'cmaes',
    ExperimentArgParams.PopulationSize.value: 5, # Very small population
    ExperimentArgParams.Sigma0.value: 1.0,
    ExperimentArgParams.Noise_strength.value: 0.01,
    
    # Internal flags (usually set by MultiExperiment, defaults here)
    ArgParams.CloseScreen.value: True
}

# Check if essential paths are set
essential_paths = [WEIGHTS_DIR, REFERENCES_DIR, CUSTOM_WEIGHTS_DIR, NATURAL_RECORDINGS_DIR]
if any(p is None or '/path/to/your/' in p for p in essential_paths):
    print("ERROR: Please edit the placeholder paths (WEIGHTS_DIR, REFERENCES_DIR, etc.) in this cell first!")
else:
    try:
        print("Creating experiment from configuration...")
        # Add the experiment title needed by from_config internals
        quickstart_config[ArgParams.ExperimentTitle.value] = StretchSqueezeExperiment.EXPERIMENT_TITLE
        
        # Create the experiment instance
        experiment = StretchSqueezeExperiment.from_config(quickstart_config)
        
        print("Running minimal demo...")
        # Run the experiment
        message = experiment.run()
        
        print("\nQuickstart demo finished successfully!")
        print(f"Results saved to: {experiment.dir}")
        print(f"Elapsed time: {message.elapsed_time:.2f} seconds")

    except FileNotFoundError as e:
        print(f"\nERROR: A required file was not found: {e}")
        print("Please double-check the paths you set in the 'User Parameters' section.")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")
        import traceback
        traceback.print_exc()